In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from rich import print
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Tuple, Dict

In [2]:
def preprocess_data_binary(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Preprocess the input DataFrame by separating features and target, and splitting the data into training and validation sets.
    Args:
        df (pd.DataFrame): The input DataFrame containing the dataset.
    Returns:
        Tuple[pd.DataFrame, pd.Series]: A tuple containing the training features (X_train), validation features (X_val), training target (y_train), and validation target (y_val).
    """
    df = df.copy()
    
    # Separate features and target
    X = df.drop(['is_benign', 'category', 'attack'], axis=1)
    y = df['is_benign']
    
    # Split the data
    X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.7, random_state=37)
    
    return X_train, X_val, y_train, y_val

In [3]:
def evaluate_model(y_true, y_pred, model_name="Model", class_names=None):
    """
    Print comprehensive model evaluation metrics with both rich text output and seaborn heatmap.
    
    Parameters:
    -----------
    y_true : array-like
        True labels
    y_pred : array-like
        Predicted labels
    model_name : str, optional
        Name of the model for display purposes
    class_names : list, optional
        List of class names for axis labels
    """
    
    # Calculate core metrics
    metrics = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, average='weighted'),
        "Recall": recall_score(y_true, y_pred, average='weighted'),
        "F1 Score": f1_score(y_true, y_pred, average='weighted')
    }
    
    # Create metrics table
    table_data = [[metric, f"{value:.5f}"] for metric, value in metrics.items()]
    table = tabulate(table_data, headers=["Metric", "Score"], tablefmt="psql")
    
    print(table)
    
    # Calculate and plot confusion matrix as heatmap
    cm = confusion_matrix(y_true, y_pred)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # Create heatmap
    plt.figure(figsize=(20, 10))
    if class_names is None:
        class_names = [f"Class {i}" for i in range(cm.shape[0])]
    
    sns.heatmap(
        cm_normalized,
        annot=True,
        fmt='.2f',
        cmap='Blues',
        xticklabels=class_names,
        yticklabels=class_names
    )
    plt.title(f'{model_name} - Normalized Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')    
    plt.tight_layout()
    plt.show()
    
    return metrics

In [ ]:
def grid_search_xgboost(df: pd.DataFrame) -> xgb.XGBClassifier:
    """
    Perform grid search analysis on XGBoost classifier with improved error handling.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input DataFrame containing the dataset
    
    Returns:
    --------
    xgb.XGBClassifier
        The best model found by grid search
    """
    # Split the data
    X_train, X_val, y_train, y_val = preprocess_data_binary(df)
    
    # Initial model
    model = xgb.XGBClassifier(
        objective='binary:logistic',
        scale_pos_weight=len(y_train[y_train==0]) / len(y_train[y_train==1]),
        random_state=42
    )
    
    # Parameter grid
    param_grid = {
        'max_depth': [3, 5, 7],
        'min_child_weight': [1, 3, 5],
        'gamma': [0, 0.1, 0.2],
        'subsample': [0.8, 0.9],
        'colsample_bytree': [0.8, 0.9]
    }
    
    
    # Perform grid search
    grid_search = GridSearchCV(
        model,
        param_grid,
        cv=5,
        scoring='roc_auc',
        n_jobs=-1,
        verbose=1,
        return_train_score=True
    )
    
    grid_search.fit(X_train, y_train)
    
    # Get predictions from best model
    y_pred = grid_search.best_estimator_.predict(X_val)
    
    # Evaluate the model
    _ = evaluate_model(y_val, y_pred, "XGBoost", ["Benign", "Malicious"])
    
    return grid_search.best_estimator_


In [ ]:
# Load your CSV file
df = pd.read_csv('/kaggle/input/dataset/train_binary.csv')

In [ ]:
best_model= grid_search_xgboost(df)
best_model.save_model('xgboost_binary.json')